# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started. Once you have completed the code you can download the notebook for making a submission.

In [ ]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize
import datetime
import os
import matplotlib.pyplot as plt

We set the random seed so that the results don't vary drastically.

In [ ]:
np.random.seed(30)
import random as rn
rn.seed(30)
from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

**data path: /home/datasets/Project_data**

In [ ]:
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())
batch_size = 64 #experiment with the batch size
### 256, 128 giving unable to allocate memory while adding layers
print(len(train_doc),len(val_doc))

663 100


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [ ]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    global frames,x,y,z
    frames = 30
    img_idx = [i for i in range(0,frames) if i%2==0]#create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size# calculate the number of batches
        x = len(img_idx)
        y,z = 100, 100 ####
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
#                     print(image.shape)
#                     plt.imshow(image/256)
#                     return

                    if image.shape[0]!=y or image.shape[1]!=z:
                        image = resize(image,[y,z], anti_aliasing=True)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        remaining_folders = len(t)%batch_size
        if remaining_folders>0:
            batch_data = np.zeros((remaining_folders,x,y,z,3)) 
            batch_labels = np.zeros((remaining_folders,5)) 
            for folder in range(remaining_folders): 
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    if image.shape[0]!=y or image.shape[1]!=z:
                        image = resize(image,[y,z], anti_aliasing=True)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels 


In [ ]:
# from skimage import data


# image = data.coins()
# image2 = data.camera()
# print(image2.shape)
# # plt.imshow(image2)
# plt.imshow(resize(image2,[1900,1900], anti_aliasing=True))
# plt.show()

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


In [ ]:
# checking images
a = generator(train_path, train_doc, batch_size)
item = next(a)
print(type(item),item[0].shape)
# item = next(a)
# print(type(item),item[0].shape)
# item = next(a)
# print(type(item),item[0].shape)
# item = next(a)
# print(type(item),item[0].shape)

Source path =  /home/datasets/Project_data/train ; batch size = 64
<class 'tuple'> (64, 15, 100, 100, 3)


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [ ]:
#######
####   Conv3d Model: Accuracy 74/83 (val), batch_size=64
#######

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation, Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import L2 #vs l2
from tensorflow.keras import optimizers

#write your model here
model = Sequential()

model.add(Conv3D(64, (3, 3, 3) , padding = "SAME",
                        input_shape=(x, y, z, 3))) #### Large batch_size might not fit params into memory
model.add(Activation('relu'))
model.add(Conv3D(32, (3, 3, 3), padding = "SAME"))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.5))

model.add(Conv3D(16, (3, 3, 3), padding = "SAME"))
model.add(Activation('relu'))
# model.add(Conv3D(16, (3, 3, 3), padding = "SAME"))
# model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.5))

# model.add(Conv3D(32, (3, 3, 3), padding = "SAME"))
# model.add(Activation('relu'))
# model.add(Conv3D(8, (3, 3, 3), padding = "SAME"))
# model.add(Activation('relu'))
# model.add(MaxPooling3D(pool_size=(1, 2, 2)))
# model.add(Dropout(0.5))

# model.add(Conv3D(32, (3, 3, 3), padding = "SAME"))
# model.add(Activation('relu'))
# model.add(Conv3D(8, (3, 3, 3), padding = "SAME"))
# model.add(Activation('relu'))
# model.add(MaxPooling3D(pool_size=(1, 2, 2)))
# model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512, kernel_regularizer = L2(0.1)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5, kernel_regularizer = L2(0.1)))  #use W_regularizer if .tf.keras.layers.core.Dense
model.add(Activation('softmax'))


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [ ]:
learning_rate = 0.001
optimiser = optimizers.Adam(lr=learning_rate, clipvalue=1.0)#write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 15, 100, 100, 64)  5248      
_________________________________________________________________
activation (Activation)      (None, 15, 100, 100, 64)  0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 15, 100, 100, 32)  55328     
_________________________________________________________________
activation_1 (Activation)    (None, 15, 100, 100, 32)  0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 7, 50, 50, 32)     0         
_________________________________________________________________
dropout (Dropout)            (None, 7, 50, 50, 32)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 7, 50, 50, 16)    

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)
# a = next(val_generator)
# print(a[0].shape,type(a[1][0][0]))

In [ ]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.3f}-{categorical_accuracy:.3f}-{val_loss:.3f}-{val_categorical_accuracy:.3f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',save_freq = "epoch")

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=3)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit` method to decide the number of next() calls it need to make.

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=True, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 64
Epoch 1/50
11/11 [==============================] - ETA: 0s - loss: 22.1696 - categorical_accuracy: 0.1946Source path =  /home/datasets/Project_data/val ; batch size = 64

Epoch 00001: saving model to model_init_2021-06-2711_18_46.352023/model-00001-22.170-0.195-18.091-0.210.h5
11/11 [==============================] - 84s 8s/step - loss: 22.1696 - categorical_accuracy: 0.1946 - val_loss: 18.0909 - val_categorical_accuracy: 0.2100
Epoch 2/50
11/11 [==============================] - ETA: 0s - loss: 15.5164 - categorical_accuracy: 0.2081
Epoch 00002: saving model to model_init_2021-06-2711_18_46.352023/model-00002-15.516-0.208-12.400-0.230.h5
11/11 [==============================] - 83s 8s/step - loss: 15.5164 - categorical_accuracy: 0.2081 - val_loss: 12.4001 - val_categorical_accuracy: 0.2300
Epoch 3/50
11/11 [==============================] - ETA: 0s - loss: 10.5544 - categorical_accuracy: 0.2127
Epoch 00003: saving mod

In [ ]:
# !ls -ltrh /home/dl_content/'Upgrad DL'/'Gesture Recognition Assignment'/model_init_2021-*
# !du -ah /home | grep -v "/$" | sort -rh | head -6 
# !rm -rf /home/.local/share/Trash/files
# !df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          10G   92K   10G   1% /
tmpfs            64M     0   64M   0% /dev
tmpfs           126G     0  126G   0% /sys/fs/cgroup
/dev/nvme0n1     10G  5.8G  4.3G  58% /datasets
tmpfs           126G  720K  126G   1% /dev/shm
tmpfs           126G   12K  126G   1% /proc/driver/nvidia
/dev/sda2       3.5T  395G  2.9T  12% /usr/bin/nvidia-smi
tmpfs            26G  4.1M   26G   1% /run/nvidia-persistenced/socket
udev            126G     0  126G   0% /dev/nvidia0
tmpfs           126G     0  126G   0% /proc/asound
tmpfs           126G     0  126G   0% /proc/acpi
tmpfs           126G     0  126G   0% /proc/scsi
tmpfs           126G     0  126G   0% /sys/firmware


### CNN-RNN Model

In [ ]:
########
####   CNN-RNN Model: Accuray 94/82(val) , batch_size = 32
########


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,TimeDistributed, LSTM, GRU, Dropout, Flatten, BatchNormalization, Activation, Conv2D,Conv3D, MaxPooling2D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import L2 #vs l2
from tensorflow.keras import optimizers

#write your model here
model = Sequential()

## TimeDistributed automaticaly incorporates x=15, when model .build

model.add(TimeDistributed(Conv2D(32, (3, 3) , padding = "SAME",
                        input_shape=(y, z, 3)))) #### Large batch_size might not fit params into memory
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(32, ( 3, 3), padding = "SAME")))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Dropout(0.5)))

# model.add(TimeDistributed(Conv2D(64, (3, 3) , padding = "SAME"))) 
# model.add(TimeDistributed(Activation('relu')))
# model.add(TimeDistributed(Conv2D(64, ( 3, 3), padding = "SAME")))
# model.add(TimeDistributed(Activation('relu')))
# model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
# model.add(TimeDistributed(Dropout(0.5)))

# model.add(TimeDistributed(Conv2D(32, (3, 3) , padding = "SAME"))) 
# model.add(TimeDistributed(Activation('relu')))
# model.add(TimeDistributed(Conv2D(32, ( 3, 3), padding = "SAME")))
# model.add(TimeDistributed(Activation('relu')))
# model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
# model.add(TimeDistributed(Dropout(0.5)))

model.add(TimeDistributed(Flatten()))
model.add(LSTM(32, 
#                input_shape = [None,x]
#                 activation="relu",
#                 dropout = 0.5
              ))    
model.add(Dropout(0.5))

## number of params - 74240 = 4 set of wts * ( 128 output  * ( 16 emb inputdim + 1 bias) +   128 output^2)

# Layer 5 - Dense FFN with Dropout
model.add(BatchNormalization())
model.add(Dense(128))
model.add(Dropout(0.5))
# model.add(BatchNormalization())

model.add(Dense(5, 
#                 kernel_initializer = weights,
                    name='output'))
model.add(Activation('softmax'))




In [ ]:
learning_rate = 0.001
optimiser = optimizers.Adam(lr=learning_rate, clipvalue=1.0)#write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.build(input_shape = [None, x ,y, z ,3]) ##### specify seq

print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 15, 100, 100, 32)  896       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 15, 100, 100, 32)  0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 15, 100, 100, 32)  9248      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 15, 100, 100, 32)  0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 15, 50, 50, 32)    0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 15, 50, 50, 32)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 15, 80000)         0

In [ ]:
batch_size = 32

In [ ]:
# train_ds = tf.data.Dataset.from_generator(train_generator.generate, output_types=(tf.float32, tf.int32))
# train_ds = train_ds.map(lambda x,y : (x,y)).prefetch(buffer_size=1000)
# train_ds = train_ds.batch(batch_size)

# val_ds = tf.data.Dataset.from_generator(val_generator.generate, output_types=(tf.float32, tf.int32))
# val_ds = val_ds.map(lambda x,y : (x,y)).prefetch(buffer_size=1000)
# # dataset = dataset.map(lambda x,y : (x,y), num_parallel_calls=NUM_THREADS).prefetch(buffer_size=1000)
# val_ds = val_ds.batch(batch_size)


In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.3f}-{categorical_accuracy:.3f}-{val_loss:.3f}-{val_categorical_accuracy:.3f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',save_freq = "epoch")

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=3)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=True, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/50
21/21 [==============================] - ETA: 0s - loss: 1.8293 - categorical_accuracy: 0.2428Source path =  /home/datasets/Project_data/val ; batch size = 32

Epoch 00001: saving model to model_init_2021-06-2718_19_46.453166/model-00001-1.829-0.243-1.604-0.180.h5
21/21 [==============================] - 70s 3s/step - loss: 1.8293 - categorical_accuracy: 0.2428 - val_loss: 1.6037 - val_categorical_accuracy: 0.1800
Epoch 2/50
21/21 [==============================] - ETA: 0s - loss: 1.6770 - categorical_accuracy: 0.2172
Epoch 00002: saving model to model_init_2021-06-2718_19_46.453166/model-00002-1.677-0.217-1.597-0.210.h5
21/21 [==============================] - 72s 3s/step - loss: 1.6770 - categorical_accuracy: 0.2172 - val_loss: 1.5970 - val_categorical_accuracy: 0.2100
Epoch 3/50
21/21 [==============================] - ETA: 0s - loss: 1.6021 - categorical_accuracy: 0.2353
Epoch 00003: saving model to model

### CNN_RNN with Transfer learning (Resnet50)

In [ ]:
###  GENERATOR to use same preprocessing used in resnet50
import tensorflow.keras as K
import tensorflow

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    global frames,x,y,z
    frames = 30
    img_idx = [i for i in range(0,frames) if i%2==0]#create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size# calculate the number of batches
        x = len(img_idx)
        y,z = 224, 224 ####
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
#                     print(image.shape)
#                     plt.imshow(image/256)
#                     return

                    if image.shape[0]!=y or image.shape[1]!=z:
                        image = tf.image.resize(image,(y,z))
#                         image = tensorflow.keras.applications.resnet50.preprocess_input(image)
#                         image = resize(image,[y,z], anti_aliasing=True)  --- Added layer for this
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = image[:,:,0]
                    batch_data[folder,idx,:,:,1] = image[:,:,1]
                    batch_data[folder,idx,:,:,2] = image[:,:,2]
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        remaining_folders = len(t)%batch_size
        if remaining_folders>0:
            batch_data = np.zeros((remaining_folders,x,y,z,3)) 
            batch_labels = np.zeros((remaining_folders,5)) 
            for folder in range(remaining_folders): 
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    if image.shape[0]!=y or image.shape[1]!=z:
                        image = tensorflow.keras.applications.resnet50.preprocess_input(image)
                        image = tf.image.resize(image,(y,z))
#                         image = resize(image,[y,z], anti_aliasing=True)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = image[:,:,0]
                    batch_data[folder,idx,:,:,1] = image[:,:,1]
                    batch_data[folder,idx,:,:,2] = image[:,:,2]
                    
                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels 


In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 100 # choose the number of epochs
print ('# epochs =', num_epochs)

a = generator(train_path, train_doc, batch_size)
item = next(a)
print(type(item),item[0].shape)

# training sequences = 663
# validation sequences = 100
# epochs = 50
Source path =  /home/datasets/Project_data/train ; batch size = 64
<class 'tuple'> (64, 15, 224, 224, 3)


In [ ]:
####
# CNN-RNN with Transfer learning, Accuracy 64/90
####

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense,TimeDistributed, LSTM, GRU, Dropout, Flatten, BatchNormalization, Activation, Conv2D,Conv3D, MaxPooling2D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import L2 #vs l2
from tensorflow.keras import optimizers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

#write your model here
model = Sequential()
# model.add(Lambda(lambda image: tensorflow.image.resize(image, (y,z))))

base_model = ResNet50(weights='imagenet', include_top=False)
# base_model.summary()    
# base_model.layers.pop() ## removing 4x4x2048 layers
# base_model = Sequential(base_model.layers[:-2])

base_model = Model(base_model.input, base_model.layers[-11].output) ########


# As we are using ResNet model only for feature extraction and not adjusting the weights
# we freeze the layers in base model
for layer in base_model.layers[:143]:
    layer.trainable = False

for layer in base_model.layers:
    print(layer.name,layer.output_shape,layer.trainable)

# base_model.summary()
# Adding our own layer 
model.add(TimeDistributed(base_model))

# model.add(TimeDistributed(Conv2D(128, (3, 3) , padding = "SAME",
#                         input_shape=(y, z, 3)))) #### Large batch_size might not fit params into memory
# model.add(TimeDistributed(Activation('relu')))
# model.add(TimeDistributed(Conv2D(128, ( 3, 3), padding = "SAME")))
# model.add(TimeDistributed(Activation('relu')))
# model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
# model.add(TimeDistributed(Dropout(0.5)))

# model.add(TimeDistributed(Conv2D(64, (3, 3) , padding = "SAME"))) 
# model.add(TimeDistributed(Activation('relu')))
# model.add(TimeDistributed(Conv2D(64, ( 3, 3), padding = "SAME")))
# model.add(TimeDistributed(Activation('relu')))
# model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
# model.add(TimeDistributed(Dropout(0.5)))

## reduce filters from 256 -> 32
# model.add(TimeDistributed(Conv2D(32, (3, 3) , padding = "SAME"))) 
# model.add(TimeDistributed(Activation('relu')))
# model.add(TimeDistributed(Conv2D(32, ( 3, 3), padding = "SAME")))
# model.add(TimeDistributed(Activation('relu')))
# model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
# model.add(TimeDistributed(Dropout(0.5)))

# model.add(pretrained)
model.add(TimeDistributed(Flatten()))
model.add(LSTM(32
#                input_shape = [None,x]
#                 activation="relu",
#                 dropout = 0.5
              ))    
model.add(Dropout(0.5))

## number of params - 74240 = 4 set of wts * ( 128 output  * ( 16 emb inputdim + 1 bias) +   128 output^2)

# Layer 5 - Dense FFN with Dropout
model.add(BatchNormalization())
model.add(Dense(128))
model.add(Dropout(0.5))
model.add(BatchNormalization())   ####### better if removed
model.add(Dense(5, 
#                 kernel_initializer = weights,
                    name='output'))
model.add(Activation('softmax'))



input_14 [(None, None, None, 3)] False
conv1_pad (None, None, None, 3) False
conv1_conv (None, None, None, 64) False
conv1_bn (None, None, None, 64) False
conv1_relu (None, None, None, 64) False
pool1_pad (None, None, None, 64) False
pool1_pool (None, None, None, 64) False
conv2_block1_1_conv (None, None, None, 64) False
conv2_block1_1_bn (None, None, None, 64) False
conv2_block1_1_relu (None, None, None, 64) False
conv2_block1_2_conv (None, None, None, 64) False
conv2_block1_2_bn (None, None, None, 64) False
conv2_block1_2_relu (None, None, None, 64) False
conv2_block1_0_conv (None, None, None, 256) False
conv2_block1_3_conv (None, None, None, 256) False
conv2_block1_0_bn (None, None, None, 256) False
conv2_block1_3_bn (None, None, None, 256) False
conv2_block1_add (None, None, None, 256) False
conv2_block1_out (None, None, None, 256) False
conv2_block2_1_conv (None, None, None, 64) False
conv2_block2_1_bn (None, None, None, 64) False
conv2_block2_1_relu (None, None, None, 64) False
c

In [ ]:
learning_rate = 0.001
optimiser = optimizers.Adam(lr=learning_rate, clipvalue=1.0)#write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.build(input_shape = [None, x ,y, z ,3]) ##### specify seq

print (model.summary())

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_50 (TimeDis (None, 15, 7, 7, 2048)    19115904  
_________________________________________________________________
time_distributed_51 (TimeDis (None, 15, 100352)        0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 32)                12849280  
_________________________________________________________________
dropout_26 (Dropout)         (None, 32)                0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 32)                128       
_________________________________________________________________
dense_9 (Dense)              (None, 128)               4224      
_________________________________________________________________
dropout_27 (Dropout)         (None, 128)             

In [ ]:
batch_size = 32

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.3f}-{categorical_accuracy:.3f}-{val_loss:.3f}-{val_categorical_accuracy:.3f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',save_freq = "epoch")

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=3)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=True, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/50
21/21 [==============================] - ETA: 0s - loss: 2.1369 - categorical_accuracy: 0.2308Source path =  /home/datasets/Project_data/val ; batch size = 32

Epoch 00001: saving model to model_init_2021-06-2816_42_42.890086/model-00001-2.137-0.231-1.643-0.160.h5
21/21 [==============================] - 107s 5s/step - loss: 2.1369 - categorical_accuracy: 0.2308 - val_loss: 1.6429 - val_categorical_accuracy: 0.1600
Epoch 2/50
21/21 [==============================] - ETA: 0s - loss: 1.8576 - categorical_accuracy: 0.2534
Epoch 00002: saving model to model_init_2021-06-2816_42_42.890086/model-00002-1.858-0.253-1.493-0.330.h5
21/21 [==============================] - 111s 5s/step - loss: 1.8576 - categorical_accuracy: 0.2534 - val_loss: 1.4926 - val_categorical_accuracy: 0.3300
Epoch 3/50
21/21 [==============================] - ETA: 0s - loss: 1.6530 - categorical_accuracy: 0.3198
Epoch 00003: saving model to mod

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=True, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/50
21/21 [==============================] - ETA: 0s - loss: 1.7505 - categorical_accuracy: 0.2383Source path =  /home/datasets/Project_data/val ; batch size = 32

Epoch 00001: saving model to model_init_2021-06-2816_42_42.890086/model-00001-1.751-0.238-1.614-0.310.h5
21/21 [==============================] - 104s 5s/step - loss: 1.7505 - categorical_accuracy: 0.2383 - val_loss: 1.6144 - val_categorical_accuracy: 0.3100
Epoch 2/50
21/21 [==============================] - ETA: 0s - loss: 1.6952 - categorical_accuracy: 0.2730
Epoch 00002: saving model to model_init_2021-06-2816_42_42.890086/model-00002-1.695-0.273-1.538-0.270.h5
21/21 [==============================] - 103s 5s/step - loss: 1.6952 - categorical_accuracy: 0.2730 - val_loss: 1.5378 - val_categorical_accuracy: 0.2700
Epoch 3/50
21/21 [==============================] - ETA: 0s - loss: 1.6232 - categorical_accuracy: 0.2715
Epoch 00003: saving model to mod

KeyboardInterrupt: 

In [ ]:
### Val accuracy is more since, drop out is not used at test/validation time , and more neurons are used during testing